In [1]:
import os
import sys
sys.path.insert(0, '../utils')
sys.path.insert(0, '..')
import logging

logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
logger = logging.getLogger('__name__')

import numpy as np
import tensorflow as tf
from tensorflow.keras.metrics import Mean
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, SGD, Nadam

from modelsTF import *
from dataGenerator import *
%reload_ext autoreload
%autoreload 2

In [10]:
# import data
CLEAN_DATA_DIR = '/home/mark/DataBank/PROBA-V-CHKPT/augmentedPatchesDir'
band = 'NIR'
X = np.load(os.path.join(CLEAN_DATA_DIR, f'TRAINpatchesLR_{band}.npy'), allow_pickle=True)
y = np.load(os.path.join(CLEAN_DATA_DIR, f'TRAINpatchesHR_{band}.npy'), allow_pickle=True)

#X = X_train.transpose((0, 3, 4, 2, 1))
#y = y_train.transpose((0, 3, 4, 2, 1)).squeeze(3)
print(f'Input shape: {X.shape} --------> Output shape: {y.shape}')

Input shape: (230720, 38, 38, 9, 1) --------> Output shape: (230720, 96, 96, 1)


In [4]:
datasetAllMean = 8818.0603
datasetAllStd = 6534.1132
modelIns = WDSRConv3D(name='patch38', band="rid", mean=datasetAllMean, std=datasetAllStd, maxShift=6)
model = modelIns.build(scale=3, numFilters=32, kernelSize=(3, 3, 3), numResBlocks=12,
                           expRate=8, decayRate=0.8, numImgLR=7, patchSizeLR=38, isGrayScale=True)

In [5]:
model.summary()

Model: "WDSRConv3D_rid_patch38"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 38, 38, 7, 1 0                                            
__________________________________________________________________________________________________
normImgLR (Lambda)              (None, 38, 38, 7, 1) 0           input_2[0][0]                    
__________________________________________________________________________________________________
mainConv1 (WeightNormalization) (None, 38, 38, 7, 32 929         normImgLR[0][0]                  
__________________________________________________________________________________________________
expConv_0 (WeightNormalization) (None, 38, 38, 7, 25 8705        mainConv1[0][0]                  
_____________________________________________________________________________

In [13]:
np.mean(X)

8818.060385164923

In [14]:
np.std(X)

6534.113214217506

In [16]:
with tf.device('/GPU:0'):
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

c = tf.matmul(a, b)
print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [2]:
strategy = tf.distribute.MirroredStrategy()

In [4]:
strategy.experimental_run_v2

<bound method Strategy.experimental_run_v2 of <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7f5a650eee80>>

In [1]:
import numpy as np

In [2]:
ab = np.ma.masked_array([1], mask=[0])

In [3]:
ab

masked_array(data=[1],
             mask=[False],
       fill_value=999999)

In [4]:
tf.distribute.experimental.MultiWorkerMirroredStrategy()

array([1])

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Adding a dimension to the array -> new shape == (28, 28, 1)
# We are doing this because the first layer in our model is a convolutional
# layer and it requires a 4D input (batch_size, height, width, channels).
# batch_size dimension will be added later on.
train_images = train_images[..., None]
test_images = test_images[..., None]

# Getting the images in [0, 1] range.
train_images = train_images / np.float32(255)
test_images = test_images / np.float32(255)